In [97]:
import src.modules.scraping as sc
import polars as pl
import re

In [98]:
sc.json_brut(sc.scraping("https://www.vinatis.com/52605-vacqueyras-vieilles-vignes-2020-domaine-sandilaure"))

c:\Users\guill\OneDrive - Université de Tours\Bureau\M2\ML\Wine-Scraping\src\modules\scraping.py:11: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 11 of the file c:\Users\guill\OneDrive - Université de Tours\Bureau\M2\ML\Wine-Scraping\src\modules\scraping.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soupe = BS(requete.text)


In [99]:
data = pl.read_json("data/vins.json")

In [100]:
df = {
    "name" : data["name"].str.strip(),
    "capacity" : data["capacity"].str.strip(),
    "price" : data["prices"][0][0].strip(),
    "price_discount_1" : data["prices"][0][1].strip(),
    "price_discount_2" : data["prices"][0][2].strip(),
    "characteristic" : data["characteristic"].str.strip(),
    "note" : data["note"].str.strip(),
    "keyword_1": data["keyword"][0][0].strip(),
    "keyword_2": data["keyword"][0][1].strip(),
    "keyword_3": data["keyword"][0][2].strip(),
    "vintage": data["informations"][0][" \xa0\xa0Millésime"].strip(),
    "grape_variety": data["informations"][0][' \xa0\xa0Goûts'].strip(),
    "taste": data["informations"][0][' \xa0\xa0Goûts'].strip(),
    "taste_type": data["informations"][0][ ' \xa0\xa0Par Goûts'].strip(),
    "eye": data["informations"][0][ " \xa0\xa0A l'oeil"].strip(),
    "nose": data["informations"][0][' \xa0\xa0Au nez'].strip(),
    "mouth": data["informations"][0][' \xa0\xa0En bouche'].strip(),
    "temperature": data["informations"][0][' \xa0\xa0Température de service'].strip(),
    "service": data["informations"][0][' \xa0\xa0Service'].strip(),
    "conservation": data["informations"][0][' \xa0\xa0Conservation'].strip(),
    "max_date": data["informations"][0][" \xa0\xa0Jusqu'à"].strip(),
    "food": data["informations"][0][' \xa0\xa0Accords mets-vin'].strip(),
    "recomandation": data["informations"][0][' \xa0\xa0Accords recommandés'].strip(),
    "picture" : data["picture"].str.strip()
}

C:\Users\guill\AppData\Local\Temp\ipykernel_8000\3420030718.py:2: DeprecationWarning: `strip` is deprecated. It has been renamed to `strip_chars`.
  "name" : data["name"].str.strip(),
C:\Users\guill\AppData\Local\Temp\ipykernel_8000\3420030718.py:3: DeprecationWarning: `strip` is deprecated. It has been renamed to `strip_chars`.
  "capacity" : data["capacity"].str.strip(),
C:\Users\guill\AppData\Local\Temp\ipykernel_8000\3420030718.py:7: DeprecationWarning: `strip` is deprecated. It has been renamed to `strip_chars`.
  "characteristic" : data["characteristic"].str.strip(),
C:\Users\guill\AppData\Local\Temp\ipykernel_8000\3420030718.py:8: DeprecationWarning: `strip` is deprecated. It has been renamed to `strip_chars`.
  "note" : data["note"].str.strip(),
C:\Users\guill\AppData\Local\Temp\ipykernel_8000\3420030718.py:25: DeprecationWarning: `strip` is deprecated. It has been renamed to `strip_chars`.
  "picture" : data["picture"].str.strip()


In [101]:
df = pl.DataFrame(df)

In [102]:
df

name,capacity,price,price_discount_1,price_discount_2,characteristic,note,keyword_1,keyword_2,keyword_3,vintage,grape_variety,taste,taste_type,eye,nose,mouth,temperature,service,conservation,max_date,food,recomandation,picture
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""VACQUEYRAS VIE…","""0,75 L""","""15,50 €""","""11,90 €""","""None""","""Vin Rouge / Rh…","""16.9/20 noté …","""Epicé""","""Charpenté""","""Maison inconto…","""2020""","""Rouge Puissant…","""Rouge Puissant…","""Puissant""","""Belle robe gre…","""Complexe et tr…","""Une bouche int…","""16-18°C""","""Ouvrir 1h avan…","""A boire et à g…","""2032""","""Charcuterie, V…","""Grillades, ent…","""https://www.vi…"


In [103]:
df = df.with_columns(
    pl.col("capacity").str.replace(" L","")
    .str.replace(",",".")
    .cast(pl.Float64)
)

In [104]:
df = df.with_columns(
    pl.col("price").str.replace(" €","")
    .str.replace(",",".")
    .cast(pl.Float64),
)

In [105]:
try:
    df = df.with_columns(
        pl.col("price_discount_1").str.replace(" €","")
        .str.replace(",",".")
        .cast(pl.Float64),
    )
except:
    pass

In [106]:
try:
    df = df.with_columns(
        pl.col("price_discount_2").str.replace(" €","")
        .str.replace(",",".")
        .cast(pl.Float64),
    )
except:
    pass

In [107]:
df = df.with_columns(
    pl.col("vintage").cast(pl.Int32)
)

In [108]:
df = df.with_columns(
    pl.col("max_date").cast(pl.Int32)
)

In [109]:
df = df.with_columns(
    pl.col("note")
)

In [110]:
df = df.with_columns(
    pl.col("note").apply(lambda x: x.split()[0], return_dtype=pl.Utf8)
    .alias("avis")
    .str.replace("/20","")
    .cast(pl.Float32),
    pl.col("note").apply(lambda x: x.split()[-2], return_dtype=pl.Utf8)
    .alias("nb_avis")
    .str.replace("/20","")
    .cast(pl.Int32)
)

df = df.drop("note")

C:\Users\guill\AppData\Local\Temp\ipykernel_8000\1290778878.py:2: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.col("note").apply(lambda x: x.split()[0], return_dtype=pl.Utf8)
C:\Users\guill\AppData\Local\Temp\ipykernel_8000\1290778878.py:6: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.col("note").apply(lambda x: x.split()[-2], return_dtype=pl.Utf8)


In [125]:
def extract_charac(text: str) -> int | None:
    """
    Extract the `year` of the wine from unstructured text using a regular expression pattern.
    """
    match = re.findall(r'[^/]+', text)
    if match:
        return match
    else:
        return None

In [128]:
df = df.with_columns(
    pl.col("characteristic").map_elements(lambda x: extract_charac(x[0])[0].alias("color"))
)

ComputeError: AttributeError: 'str' object has no attribute 'alias'

In [121]:
extract_year(df["characteristic"][0])

['Vin Rouge ', ' Rhône ', ' Vacqueyras AOC ', ' 14 % vol']

In [112]:
df["characteristic"][0]

'Vin Rouge / Rhône / Vacqueyras AOC / 14 % vol'

In [95]:
lf = df.lazy()

In [96]:
lf

<LazyFrame [25 cols, {"name": Utf8 … "nb_avis": Int32}] at 0x2326E767FD0>